In [1]:
import pandas as pd
import numpy as np

In [2]:
names = ['age', 'work-class', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
         'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
         'target']
adult_data = pd.read_csv('../Datasets/adult.data', header=None)
adult_data.columns = names

adult_test = pd.read_csv('../Datasets/adult.test', header=None)
adult_test.columns = names

adult_data = adult_data.replace({' ?':np.nan}).dropna()
adult_test = adult_test.replace({' ?':np.nan}).dropna()
adult_data.head()

In [4]:
# adult_data.to_csv('../Datasets/adult_train.csv')
# adult_test.to_csv('../Datasets/adult_test.csv')

In [30]:
categorial_attr = ['work-class', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
df_adult = adult_data[categorial_attr]
df_adult = pd.get_dummies(df_adult, drop_first=True)
target = pd.get_dummies(adult_data['target'], drop_first=True)
df_adult.head()

,work-class_ Local-gov,work-class_ Private,work-class_ Self-emp-inc,work-class_ Self-emp-not-inc,work-class_ State-gov,work-class_ Without-pay,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,...,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male
0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [31]:
df_test = adult_test[categorial_attr]
df_test = pd.get_dummies(df_test, drop_first=True)
test_target = pd.get_dummies(adult_test['target'], drop_first=True)

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
log_reg = LogisticRegression(solver='liblinear', multi_class='ovr')
log_reg.fit(df_adult.values, target.values)

original_acc = accuracy_score(log_reg.predict(df_adult.values), target)
# log_reg.score(df_adult.values, target)

/home/xma/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
# we flip the values of "sex"
df_sex = df_adult.copy()
df_sex['sex_ Male'] = df_sex['sex_ Male'].apply(lambda x: x^1)
sex_acc = accuracy_score(log_reg.predict(df_sex.values), target) # accuracy reduces 0.02
print('flip sex values, accuracy reduction: ', original_acc-sex_acc)

flip sex values, accuracy reduction:  0.023672170280485405


In [24]:
# flip the values of 'race black'
df_race = df_adult.copy()
df_race['race_ Black'] = df_race['race_ Black'].apply(lambda x: x^1)
race_acc = accuracy_score(log_reg.predict(df_race.values), target) # accuracy reduces 0.003
print('flip race black accuracy reduction: ', original_acc-race_acc)

flip race black accuracy reduction:  0.0029175784099197966


In [25]:
from sklearn import svm
svm_clf = svm.SVC(gamma='scale')
svm_clf.fit(df_adult.values, target.values)
svm_accuracy = svm_clf.score(df_adult.values, target)

/home/xma/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
svm_sex_acc = accuracy_score(svm_clf.predict(df_sex.values), target)
svm_race_acc = accuracy_score(svm_clf.predict(df_race.values), target)
print('flip sex values, accuracy reduciton: ', svm_accuracy - svm_sex_acc)
print('flip race black values, accuracy reduciton: ', svm_accuracy - svm_race_acc)

flip sex values, accuracy reduciton:  0.0019229494065380326
flip race black values, accuracy reduciton:  0.0010609376036071483
